# DS-SF-30 | Unit Project 3: Machine Learning Modeling

In this project, you will perform a logistic regression on the admissions data we've been working with in Unit Projects 1 and 2.

In [2]:
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)
pd.set_option('display.notebook_repr_html', True)

import statsmodels.formula.api as smf

from sklearn import linear_model

In [3]:
df = pd.read_csv(os.path.join('..', '..', 'dataset', 'dataset-ucla-admissions.csv'))
df.dropna(inplace = True)

df

,admit,gre,gpa,prestige
0,0,380.0,3.61,3.0
1,1,660.0,3.67,3.0
2,1,800.0,4.00,1.0
3,1,640.0,3.19,4.0
4,0,520.0,2.93,4.0
...,...,...,...,...
395,0,620.0,4.00,2.0
396,0,560.0,3.04,3.0
397,0,460.0,2.63,2.0
398,0,700.0,3.65,2.0


## Part A.  Frequency Table

> ### Question 1.  Create a frequency table for `prestige` and whether an applicant was admitted.

In [4]:
freq_prest = df[['prestige','admit']].groupby(['prestige']).sum()
freq_prest_total = df[['prestige','admit']].groupby(['prestige']).count()# TODO
freq_prest_total.rename(columns={'admit':'total'},inplace=True)
freq_prest = freq_prest.join(freq_prest_total)
freq_prest['no_admit'] = freq_prest['total'] - freq_prest['admit']
freq_prest['admit_pct'] = freq_prest['admit']/freq_prest['total']# TODO

## Part B.  Variable Transformations

> ### Question 2.  Create a one-hot encoding for `prestige`.

In [5]:
cs = pd.get_dummies(df.prestige, prefix = 'prestige')# TODO

> ### Question 3.  How many of these binary variables do we need for modeling?

Answer: just 3

> ### Question 4.  Why are we doing this?

Answer: To use turn a single categorical value in to ones we can use in a model

> ### Question 5.  Add all these binary variables in the dataset and remove the now redundant `prestige` feature.

In [6]:
df = df.join(cs)

In [7]:
df.drop('prestige',axis=1,inplace=True)# TODO

## Part C.  Hand calculating odds ratios

Let's develop our intuition about expected outcomes by hand calculating odds ratios.

> ### Question 6.  Create a frequency table for `prestige = 1` and whether an applicant was admitted.

In [8]:
freq_prest # TODO

,admit,total,no_admit,admit_pct
prestige,,,,
1.0,33,61,28,0.540984
2.0,53,148,95,0.358108
3.0,28,121,93,0.231405
4.0,12,67,55,0.179104


> ### Question 7.  Use the frequency table above to calculate the odds of being admitted to graduate school for applicants that attended the most prestigious undergraduate schools.

In [31]:
one_prob = (freq_prest.admit[1]*1.)/(freq_prest.total[1]*1.)# 54.1%
print one_prob / (1-one_prob), '#1 odds'

1.17857142857 #1 odds


> ### Question 8.  Now calculate the odds of admission for undergraduates who did not attend a #1 ranked college.

In [32]:
other_prob = (freq_prest.admit[2:]*1.).sum()/(freq_prest.total[2:]*1.).sum()# TODO
print other_prob / (1-other_prob), 'not #1 odds'

0.382716049383 not #1 odds


> ### Question 9.  Finally, what's the odds ratio?

In [35]:
print (one_prob / (1-one_prob)) / (other_prob / (1-other_prob)), 'odds ratio'

3.07949308756 odds ratio


> ### Question 10.  Write this finding in a sentence.

Answer: The odds of getting admitted from a tier 1 school vs. other schools is 3.1:1.

> ### Question 11.  Use the frequency table above to calculate the odds of being admitted to graduate school for applicants that attended the least prestigious undergraduate schools.  Then calculate their odds ratio of being admitted to UCLA.  Finally, write this finding in a sentence.

In [36]:
freq_prest# TODO

,admit,total,no_admit,admit_pct
prestige,,,,
1.0,33,61,28,0.540984
2.0,53,148,95,0.358108
3.0,28,121,93,0.231405
4.0,12,67,55,0.179104


In [84]:
four_odd = freq_prest.admit_pct[4]/(1-freq_prest.admit_pct[4])

In [87]:
all_odd = (freq_prest.admit_pct[:3]*1.).sum()/(1-freq_prest.admit_pct[:3]*1.).sum()

In [88]:
four_odd / all_odd

0.36080746146797876

Answer: there is just .36:1 odds of being amitted to UCLA compared to all students.

## Part C. Analysis using `statsmodels`

> ### Question 12.  Fit a logistic regression model predicting admission into UCLA using `gre`, `gpa`, and the `prestige` of the undergraduate schools.  Use the highest prestige undergraduate schools as your reference point.

In [89]:
df.columns

Index([u'admit', u'gre', u'gpa', u'prestige_1', u'prestige_2', u'prestige_3',
       u'prestige_4'],
      dtype='object')

In [90]:
df.rename(columns={'prestige_1.0':'prestige_1', 'prestige_2.0':'prestige_2',
       'prestige_3.0':'prestige_3', 'prestige_4.0':'prestige_4'},inplace=True)

In [91]:
train_df = df.sample(frac = .6, random_state = 0)
test_df = df.drop(train_df.index)

In [109]:
#names_X = ['gre', 'gpa', 'prestige_1.0',
#    'prestige_2.0', 'prestige_3.0']
# fit the model
result = model.fit()model = smf.Logit.from_formula('admit ~ gre + gpa + prestige_1 + prestige_2 + prestige_3', data=df) # TODO

result.summary()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Tue, 07 Feb 2017   Pseudo R-squ.:                 0.08166
Time:                        15:34:41   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     -5.4303      1.140     -4.764      0.000        -7.664    -3.196
gre            0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa            0.7793      0.333      2.344      0.019         0.128     1.431
prestige_1     1.5534      0.417      3.721      0.000         0.735     2.372
prestige_2     0.8733      0.367      2.378      0.017         0.153     1.593
prestige_3     0.2147      0.393      0.547      0.584        -0.555     0.984
==============================================================================
"""

> ### Question 14.  What are the odds ratios of the different features and their 95% confidence intervals?

gre odds: 0.0022 intervale:7.44e-05 0.004

gpa odds:0.7793 interval: 0.128 1.431

prestige_1	odds: 1.5534 interval: 0.735 2.372

prestige_2	odds: 0.8733 inteval: 0.153 1.593

prestige_3	odds: 0.2147 interval:-0.555 0.984




> ### Question 15.  Interpret the odds ratio for `prestige = 2`.

The odds that people who went to 2 prestige schools are admitted are is 87% more than prestige 4.

> ### Question 16.  Interpret the odds ratio of `gpa`.

Answer: Every extra GPA point increases the odds of getting in by 77.9% (seems wrong).

> ### Question 17.  Assuming a student with a GRE of 800 and a GPA of 4.  What is his/her probability of admission  if he/she come from a tier-1, tier-2, tier-3, or tier-4 undergraduate school?

In [110]:
names_X = ['gre', 'gpa', 'prestige_1',
    'prestige_2', 'prestige_3']

In [116]:
predict_X = pd.DataFrame([[800, 4, 1, 0, 0],[800, 4, 0, 1, 0],[800, 4, 0, 0, 1],[800, 4, 0, 0, 0]],columns=names_X)

result.predict(predict_X)
#print model.predict_proba(predict_X)# TODO

array([ 0.73403998,  0.58299512,  0.41983282,  0.36860803])

Answer:1 : 0.73403998, 2: 0.58299512, 3: 0.41983282, 4: 0.36860803

## Part D. Moving the model from `statsmodels` to `sklearn`

> ### Question 18.  Let's assume we are satisfied with our model.  Remodel it (same features) using `sklearn`.  When creating the logistic regression model with `LogisticRegression(C = 10 ** 2)`.

In [9]:
train_df = df.sample(frac = .6, random_state = 0)
test_df = df.drop(train_df.index)
def X_c(df):
    X = df[ names_X ]
    c = df.admit
    return X, c

train_X, train_c = X_c(train_df)
test_X, test_c = X_c(test_df)# TODO
# TODO

NameError: global name 'names_X' is not defined

> ### Question 19.  What are the odds ratios for the different variables and how do they compare with the odds ratios calculated with `statsmodels`?

In [118]:
model = linear_model.LogisticRegression().\
    fit(train_X, train_c)

print model.intercept_
print model.coef_# TODO
print 'training misclassification =', 1 - model.score(train_X, train_c)
print 'testing  misclassification =', 1 - model.score(test_X, test_c)# TODO
print zip(names_X, np.exp(model.coef_[0]) - 1)# TODO

[-2.23114167]
[[ 0.00251056 -0.05602435  1.28005515  0.41842339 -0.05567949]]
training misclassification = 0.289915966387
testing  misclassification = 0.301886792453
[('gre', 0.0025137180344876331), ('gpa', -0.054483887677666765), ('prestige_1', 2.5968380939818267), ('prestige_2', 0.51956390762323434), ('prestige_3', -0.054157756944836266)]


Answer:

> ### Question 20.  Again, assuming a student with a GRE of 800 and a GPA of 4.  What is his/her probability of admission  if he/she come from a tier-1, tier-2, tier-3, or tier-4 undergraduate school?

In [119]:
#names_X = ['gre', 'gpa', 'prestige_1.0',
#    'prestige_2.0', 'prestige_3.0']
predict_X = [ [800, 4, 1, 0, 0]]

print model.predict(predict_X)
print model.predict_proba(predict_X)# TODO

[1]
[[ 0.30295468  0.69704532]]


In [120]:
predict_X = [ [800, 4, 0, 1, 0]]

print model.predict(predict_X)
print model.predict_proba(predict_X)# TODO

[0]
[[ 0.50709066  0.49290934]]


In [121]:
predict_X = [ [800, 4, 0, 0, 1]]

print model.predict(predict_X)
print model.predict_proba(predict_X)# TODO

[0]
[[ 0.62303901  0.37696099]]


In [122]:
predict_X = [ [800, 4, 0, 0, 0]]

print model.predict(predict_X)
print model.predict_proba(predict_X)# TODO

[0]
[[ 0.60987526  0.39012474]]


Answer: